In [9]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.appName("spark").getOrCreate()

# create the Spark Context
sc = spark.sparkContext

/home/ljj0512/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


## 02 check the number of genres

In [10]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT genres FROM Korean_book")
check = cur.fetchall()
print(check[0:3])
conn.commit()
conn.close()


[('국내도서,소설,한국소설,역사/대하소설,장르소설,역사소설',), ('국내도서,소설,한국소설,한국소설일반',), ('국내도서,소설,한국소설,한국소설일반',)]


In [11]:
rdd = sc.parallelize(check).map(lambda line: line[0].split(","))
print(rdd.count())
rdd = rdd.flatMap(lambda x: x).map(lambda x: (x,1))
print(rdd.take(5))
print(rdd.count())


44945
[('국내도서', 1), ('소설', 1), ('한국소설', 1), ('역사/대하소설', 1), ('장르소설', 1)]
245812


In [14]:
rdd = rdd.reduceByKey(lambda x, y: x+y).sortBy(lambda x: -x[1])
print(rdd.take(5))
print("total genres:",rdd.count()-1) # <-- (-1)은 국내도서를 없애기 위해

[('국내도서', 44945), ('인문', 7859), ('대학교재', 7242), ('경제/경영', 5377), ('외국어', 4332)]
total genres: 2861


## pracitice

In [4]:
temp = sc.parallelize([('이재정', '박희윤'), ("김민형", "이재정"), ("이재정", "박희윤")])
temp.take(1)
temp1 = temp.distinct()
for i in temp1.collect():
    print(i)


박희윤
김민형
이상원
이재정


In [34]:
s = set({('이재정', '박희윤')})
x = ('이재정', '박희윤')
print(s)
print(x)
if x in s:
    print("already")
else:
    print("not in set")

{('이재정', '박희윤')}
('이재정', '박희윤')
already


In [25]:
x = "asdfstringasdf"
y = "stirng"
if y in x:
    print("true")
else:
    print("false")

false
